In [ ]:
# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# Cell 2: Imports and Setup
import gradio as gr
import json
import time
from typing import List, Tuple, Dict, Any
from dataclasses import dataclass
import yaml
from pathlib import Path

# Mock imports for demonstration (replace with actual implementations)
# from shared_utils.adapters.llm_adapter import LLMAdapter
# from shared_utils.rag.retriever import Retriever
# from shared_utils.agents.orchestrator import Orchestrator
# from shared_utils.ui.game_engine import TextAdventureEngine

In [ ]:
# Cell 3: Mode Handler Classes
@dataclass
class ChatConfig:
    model_id: str = "Qwen/Qwen2.5-7B-Instruct"
    max_tokens: int = 512
    temperature: float = 0.7
    system_prompt: str = "你是一個有用的AI助理，請用繁體中文回答。"


@dataclass
class RAGConfig:
    index_path: str = "indices/general.faiss"
    top_k: int = 5
    use_reranker: bool = True
    citation_format: str = "brackets"


@dataclass
class AgentsConfig:
    max_iterations: int = 3
    timeout_seconds: int = 180
    enable_tools: List[str] = None

    def __post_init__(self):
        if self.enable_tools is None:
            self.enable_tools = ["web_search", "calculator"]


@dataclass
class GameConfig:
    world_kb: str = "data/world_kb/"
    save_path: str = "outs/game_saves/"
    difficulty: str = "normal"
    narrative_style: str = "adventure"


class ModeHandler:
    """Base class for different UI modes"""

    def __init__(self, config):
        self.config = config
        self.history = []

    def process(self, message: str, history: List) -> Tuple[str, List]:
        """Process user input and return response with updated history"""
        raise NotImplementedError

    def reset(self):
        """Reset mode state"""
        self.history = []


class ChatHandler(ModeHandler):
    def __init__(self, config: ChatConfig):
        super().__init__(config)
        # self.llm = LLMAdapter(config.model_id, backend="transformers")

    def process(self, message: str, history: List) -> Tuple[str, List]:
        """Simple chat processing"""
        # Mock implementation
        response = f"[Chat模式] 收到訊息：{message}\n這是一個模擬回應。在實際實現中，這裡會呼叫 LLM。"

        # Update history
        new_history = history + [[message, response]]
        return response, new_history


class RAGHandler(ModeHandler):
    def __init__(self, config: RAGConfig):
        super().__init__(config)
        # self.retriever = Retriever(config.index_path)
        # self.llm = LLMAdapter()

    def process(self, message: str, history: List) -> Tuple[str, List]:
        """RAG-enhanced processing with citations"""
        # Mock retrieval
        mock_sources = ["文件A.pdf - 第3頁", "文件B.md - 第1節", "文件C.txt - 第45行"]

        response = f"[RAG模式] 根據檢索到的資料回答：{message}\n\n"
        response += "基於相關文件，這個問題的答案是...\n\n"
        response += "**來源引用：**\n"
        for i, source in enumerate(mock_sources[:2], 1):
            response += f"[{i}] {source}\n"

        new_history = history + [[message, response]]
        return response, new_history


class AgentsHandler(ModeHandler):
    def __init__(self, config: AgentsConfig):
        super().__init__(config)
        # self.orchestrator = Orchestrator(config)
        self.current_step = 0

    def process(self, message: str, history: List) -> Tuple[str, List]:
        """Multi-agent orchestrated processing"""
        self.current_step += 1

        # Mock agent workflow
        steps = [
            f"🔍 Researcher: 正在研究「{message}」相關資訊...",
            f"📋 Planner: 制定回答大綱...",
            f"✍️ Writer: 撰寫詳細回應...",
            f"🔍 Reviewer: 檢查事實性與引用...",
        ]

        response = f"[Agents模式] 多代理協作處理中...\n\n"
        for i, step in enumerate(steps, 1):
            response += f"步驟 {i}: {step}\n"

        response += f"\n✅ 協作完成！基於多重驗證的回答：針對「{message}」的問題..."

        new_history = history + [[message, response]]
        return response, new_history


class GameHandler(ModeHandler):
    def __init__(self, config: GameConfig):
        super().__init__(config)
        # self.game_engine = TextAdventureEngine(config)
        self.game_state = {
            "location": "森林入口",
            "hp": 100,
            "inventory": ["木劍", "治療藥水"],
            "chapter": 1,
        }

    def process(self, message: str, history: List) -> Tuple[str, List]:
        """Text adventure game processing"""
        # Mock game logic
        response = f"[遊戲模式] 🎮 文字冒險\n\n"
        response += f"📍 當前位置：{self.game_state['location']}\n"
        response += f"❤️ 生命值：{self.game_state['hp']}/100\n"
        response += f"🎒 背包：{', '.join(self.game_state['inventory'])}\n\n"

        # Process player action
        if "前進" in message or "移動" in message:
            response += "你沿著小徑前進，來到了一片開闊的草地...\n"
            self.game_state["location"] = "草地"
        elif "攻擊" in message or "戰鬥" in message:
            response += "你舉起木劍，向敵人發起攻擊！\n造成了15點傷害。"
        else:
            response += f"你說：「{message}」\n\n"
            response += (
                "**可用動作：**\n- 前進到下個區域\n- 檢查背包\n- 攻擊敵人\n- 使用物品"
            )

        new_history = history + [[message, response]]
        return response, new_history

In [ ]:
# Cell 4: UI Layout Design
def create_mode_interface():
    """Create the main interface with four modes"""

    # Initialize handlers
    chat_handler = ChatHandler(ChatConfig())
    rag_handler = RAGHandler(RAGConfig())
    agents_handler = AgentsHandler(AgentsConfig())
    game_handler = GameHandler(GameConfig())

    # Store handlers in a dict for easy access
    handlers = {
        "Chat": chat_handler,
        "RAG": rag_handler,
        "Agents": agents_handler,
        "Game": game_handler,
    }

    with gr.Blocks(
        title="ragent-text-lab: 四模式整合介面", theme=gr.themes.Soft()
    ) as demo:

        # Header
        gr.Markdown("# 🤖 ragent-text-lab: Chat/RAG/Agents/Game 整合介面")

        # Mode selector
        mode_selector = gr.Radio(
            choices=["Chat", "RAG", "Agents", "Game"],
            value="Chat",
            label="選擇模式",
            info="切換不同的AI互動模式",
        )

        # Main chat interface
        with gr.Row():
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(
                    label="對話視窗", height=500, show_copy_button=True
                )

                with gr.Row():
                    msg_input = gr.Textbox(
                        placeholder="輸入您的訊息...", scale=4, show_label=False
                    )
                    send_btn = gr.Button("發送", variant="primary")
                    clear_btn = gr.Button("清除", variant="secondary")

            # Mode-specific control panels
            with gr.Column(scale=1):
                gr.Markdown("### 🔧 控制面板")

                # Chat mode controls
                with gr.Group(visible=True) as chat_controls:
                    gr.Markdown("**Chat 模式設定**")
                    chat_temp = gr.Slider(0.1, 1.0, 0.7, label="Temperature")
                    chat_max_tokens = gr.Slider(50, 1000, 512, label="Max Tokens")
                    chat_system = gr.Textbox(
                        value="你是一個有用的AI助理", label="System Prompt", lines=2
                    )

                # RAG mode controls
                with gr.Group(visible=False) as rag_controls:
                    gr.Markdown("**RAG 模式設定**")
                    rag_top_k = gr.Slider(1, 20, 5, label="檢索數量 (Top-K)")
                    rag_reranker = gr.Checkbox(True, label="啟用重排序")
                    rag_index = gr.Dropdown(
                        choices=["general", "tech", "edu"],
                        value="general",
                        label="知識庫",
                    )

                # Agents mode controls
                with gr.Group(visible=False) as agents_controls:
                    gr.Markdown("**Agents 模式設定**")
                    agents_max_iter = gr.Slider(1, 10, 3, label="最大迭代次數")
                    agents_timeout = gr.Slider(30, 300, 180, label="超時時間(秒)")
                    agents_tools = gr.CheckboxGroup(
                        choices=["web_search", "calculator", "file_lookup"],
                        value=["web_search", "calculator"],
                        label="啟用工具",
                    )

                # Game mode controls
                with gr.Group(visible=False) as game_controls:
                    gr.Markdown("**Game 模式設定**")
                    game_difficulty = gr.Radio(
                        choices=["easy", "normal", "hard"], value="normal", label="難度"
                    )
                    game_style = gr.Dropdown(
                        choices=["adventure", "mystery", "sci-fi"],
                        value="adventure",
                        label="敘事風格",
                    )
                    game_save_btn = gr.Button("保存遊戲", size="sm")
                    game_load_btn = gr.Button("載入遊戲", size="sm")

        # Status display
        status_display = gr.Textbox(
            label="狀態", value="就緒 - Chat 模式", interactive=False
        )

        # Cell 5: Mode Switching Logic
        def switch_mode(mode):
            """Handle mode switching with proper UI updates"""
            # Control panel visibility
            chat_vis = mode == "Chat"
            rag_vis = mode == "RAG"
            agents_vis = mode == "Agents"
            game_vis = mode == "Game"

            # Reset current handler
            if mode in handlers:
                handlers[mode].reset()

            status_msg = f"已切換到 {mode} 模式"

            return (
                gr.update(visible=chat_vis),  # chat_controls
                gr.update(visible=rag_vis),  # rag_controls
                gr.update(visible=agents_vis),  # agents_controls
                gr.update(visible=game_vis),  # game_controls
                [],  # clear chatbot
                status_msg,  # update status
            )

        def process_message(message, history, mode):
            """Process user message based on current mode"""
            if not message.strip():
                return history, ""

            # Get appropriate handler
            handler = handlers.get(mode)
            if not handler:
                response = f"錯誤：未知模式 {mode}"
                return history + [[message, response]], ""

            # Process with handler
            try:
                response, new_history = handler.process(message, history)
                return new_history, ""  # Clear input
            except Exception as e:
                error_response = f"處理錯誤：{str(e)}"
                return history + [[message, error_response]], ""

        def clear_chat():
            """Clear chat history"""
            return [], "對話已清除"

        # Cell 6: Event Binding
        # Mode switching
        mode_selector.change(
            switch_mode,
            inputs=[mode_selector],
            outputs=[
                chat_controls,
                rag_controls,
                agents_controls,
                game_controls,
                chatbot,
                status_display,
            ],
        )

        # Message processing
        send_btn.click(
            process_message,
            inputs=[msg_input, chatbot, mode_selector],
            outputs=[chatbot, msg_input],
        )

        msg_input.submit(
            process_message,
            inputs=[msg_input, chatbot, mode_selector],
            outputs=[chatbot, msg_input],
        )

        # Clear chat
        clear_btn.click(clear_chat, outputs=[chatbot, status_display])

        # Game controls (mock handlers)
        game_save_btn.click(
            lambda: "遊戲已保存至 save_001.json", outputs=[status_display]
        )

        game_load_btn.click(
            lambda: "遊戲已從 save_001.json 載入", outputs=[status_display]
        )

    return demo

In [ ]:
# Cell 7: Launch Application
if __name__ == "__main__":
    demo = create_mode_interface()

    # Launch with custom settings
    demo.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=False,
        debug=True,
        show_error=True,
    )

In [ ]:
# Cell 8: Smoke Test
def test_mode_switching():
    """Test basic mode switching functionality"""

    # Test handler initialization
    chat_config = ChatConfig()
    rag_config = RAGConfig()
    agents_config = AgentsConfig()
    game_config = GameConfig()

    handlers = {
        "Chat": ChatHandler(chat_config),
        "RAG": RAGHandler(rag_config),
        "Agents": AgentsHandler(agents_config),
        "Game": GameHandler(game_config),
    }

    # Test each mode
    test_message = "測試訊息"
    test_history = []

    results = {}
    for mode_name, handler in handlers.items():
        try:
            response, new_history = handler.process(test_message, test_history)
            results[mode_name] = {
                "success": True,
                "response_length": len(response),
                "history_updated": len(new_history) > len(test_history),
            }
        except Exception as e:
            results[mode_name] = {"success": False, "error": str(e)}

    print("🧪 Mode Switching Smoke Test Results:")
    print("=" * 50)
    for mode, result in results.items():
        status = "✅ PASS" if result["success"] else "❌ FAIL"
        print(f"{mode:10} | {status}")
        if result["success"]:
            print(f"           | Response: {result['response_length']} chars")
            print(
                f"           | History: {'Updated' if result['history_updated'] else 'No change'}"
            )
        else:
            print(f"           | Error: {result['error']}")
        print()

    return all(r["success"] for r in results.values())


# Run smoke test
test_passed = test_mode_switching()
print(f"\n🎯 Overall Test Status: {'✅ ALL PASS' if test_passed else '❌ SOME FAILED'}")

# Usage Notes
print("\n" + "=" * 60)
print("🚀 使用說明 (Usage Notes)")
print("=" * 60)
print("1. 啟動後造訪 http://localhost:7860")
print("2. 使用上方的模式選擇器切換 Chat/RAG/Agents/Game")
print("3. 每個模式都有對應的控制面板可調整參數")
print("4. 輸入訊息後點擊發送或按 Enter")
print("5. Game 模式支援特殊命令：前進、攻擊、檢查等")
print("\n💡 下一步：")
print("- 實際整合各階段開發的真實模組")
print("- 添加模式間的狀態持久化")
print("- 實現更豐富的遊戲互動邏輯")
print("- 優化 UI 響應性與錯誤處理")

In [ ]:
# 簡化版煙霧測試 - 可直接在 notebook 中執行
def quick_smoke_test():
    """快速驗證四模式基本功能"""

    configs = {
        "Chat": ChatConfig(),
        "RAG": RAGConfig(),
        "Agents": AgentsConfig(),
        "Game": GameConfig(),
    }

    test_input = "你好，請介紹一下自己"

    for mode_name, config in configs.items():
        print(f"🧪 Testing {mode_name} mode...")

        if mode_name == "Chat":
            handler = ChatHandler(config)
        elif mode_name == "RAG":
            handler = RAGHandler(config)
        elif mode_name == "Agents":
            handler = AgentsHandler(config)
        else:  # Game
            handler = GameHandler(config)

        response, history = handler.process(test_input, [])

        print(f"✅ {mode_name}: {len(response)} chars response")
        print(f"   History entries: {len(history)}")
        print()


quick_smoke_test()